# Polar coordinates with jaxfun

Just some tests to illustrate curvilinear coordinates.


In [1]:
# ruff: noqa: F405
import numpy as np
import sympy as sp

from jaxfun.coordinates import get_CoordSys
from jaxfun.galerkin import (
    FunctionSpace,
    Legendre,
    TensorProductSpace,
    TestFunction,
    TrialFunction,
)
from jaxfun.operators import Cross, Curl, Div, Dot, Grad
from jaxfun.utils.common import Domain

# system = 'polar'
system = "cylindrical"
r, theta, z = sp.symbols("r,theta,z", real=True, positive=True)

if system == "polar":
    C = get_CoordSys("C", sp.Lambda((r, theta), (r * sp.cos(theta), r * sp.sin(theta))))
    R = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0.5, 1),
        name="R",
        fun_str="phi",
    )
    T = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0, np.pi),
        name="T",
        fun_str="psi",
    )
    P = TensorProductSpace((R, T), system=C, name="P")

elif system == "cylindrical":
    C = get_CoordSys(
        "C", sp.Lambda((r, theta, z), (r * sp.cos(theta), r * sp.sin(theta), z))
    )
    R = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0.5, 1),
        name="R",
        fun_str="phi",
    )
    T = FunctionSpace(
        20,
        Legendre.Legendre,
        bcs={"left": {"D": 0}, "right": {"D": 0}},
        domain=Domain(0, np.pi),
        name="T",
        fun_str="psi",
    )
    Z = FunctionSpace(20, Legendre.Legendre, domain=Domain(0, 1), name="Z", fun_str="L")
    P = TensorProductSpace((R, T, Z), system=C, name="P")

In [2]:
u = TrialFunction(P, name="u")
du = Div(Grad(u))
du

Div(Grad(u(x, y, z; P)))

In [3]:
du.doit()

phi_l(r)*psi_m(theta)*Derivative(L_n(z), (z, 2)) + (r*L_n(z)*psi_m(theta)*Derivative(phi_l(r), (r, 2)) + L_n(z)*psi_m(theta)*Derivative(phi_l(r), r))/r + L_n(z)*phi_l(r)*Derivative(psi_m(theta), (theta, 2))/r**2

In [4]:
from sympy import srepr

srepr(du)

"Div(Grad(TrialFunction(x, y, z, Symbol('P'))))"

In [5]:
du = sp.Add.fromiter(C._parent.base_scalars()) * Div(Grad(u))
du

(x + y + z)*Div(Grad(u(x, y, z; P)))

In [6]:
srepr(du)

"Mul(Add(x, y, z), Div(Grad(TrialFunction(x, y, z, Symbol('P')))))"

In [7]:
du.doit()

(r*sin(theta) + r*cos(theta) + z)*(phi_l(r)*psi_m(theta)*Derivative(L_n(z), (z, 2)) + (r*L_n(z)*psi_m(theta)*Derivative(phi_l(r), (r, 2)) + L_n(z)*psi_m(theta)*Derivative(phi_l(r), r))/r + L_n(z)*phi_l(r)*Derivative(psi_m(theta), (theta, 2))/r**2)

In [8]:
Grad(u).doit()

(L_n(z)*psi_m(theta)*Derivative(phi_l(r), r))*C.b_r + (L_n(z)*phi_l(r)*Derivative(psi_m(theta), theta)/r**2)*C.b_theta + (phi_l(r)*psi_m(theta)*Derivative(L_n(z), z))*C.b_z

In [9]:
from jaxfun.galerkin.arguments import ScalarFunction

g = ScalarFunction("g", C)
g

g(x, y, z)

In [10]:
G = g.doit()
G

G(r, theta, z)

In [11]:
Div(Grad(g))

Div(Grad(g(x, y, z)))

In [12]:
Div(Grad(g)).doit()

Derivative(G(r, theta, z), (z, 2)) + (r*Derivative(G(r, theta, z), (r, 2)) + Derivative(G(r, theta, z), r))/r + Derivative(G(r, theta, z), (theta, 2))/r**2

In [13]:
P.tensorname

'R⊗T⊗Z'

In [14]:
from jaxfun.galerkin import VectorTensorProductSpace

V = VectorTensorProductSpace(P, name="V")
V.tensorname

'P×P×P'

In [15]:
V.name

'V'

In [16]:
v = TestFunction(V, name="v")
v

v(x, y, z; V)

In [17]:
v.doit()

(L_k^{(0)}(z)*phi_i^{(0)}(r)*psi_j^{(0)}(theta))*C.b_r + (L_k^{(1)}(z)*phi_i^{(1)}(r)*psi_j^{(1)}(theta))*C.b_theta + (L_k^{(2)}(z)*phi_i^{(2)}(r)*psi_j^{(2)}(theta))*C.b_z

In [18]:
Dot(v, v).doit()

r**2*L_k^{(1)}(z)**2*phi_i^{(1)}(r)**2*psi_j^{(1)}(theta)**2 + L_k^{(2)}(z)**2*phi_i^{(2)}(r)**2*psi_j^{(2)}(theta)**2 + L_k^{(0)}(z)**2*phi_i^{(0)}(r)**2*psi_j^{(0)}(theta)**2

In [19]:
if C.dims == 3:
    c = Cross(v, v).doit()
    display(c)

(-r*L_k^{(2)}(z)*L_k^{(1)}(z)*phi_i^{(2)}(r)*phi_i^{(1)}(r)*psi_j^{(2)}(theta)*psi_j^{(1)}(theta) + r*L_k^{(1)}(z)*L_k^{(2)}(z)*phi_i^{(1)}(r)*phi_i^{(2)}(r)*psi_j^{(1)}(theta)*psi_j^{(2)}(theta))*C.b_r + (-L_k^{(0)}(z)*L_k^{(2)}(z)*phi_i^{(0)}(r)*phi_i^{(2)}(r)*psi_j^{(0)}(theta)*psi_j^{(2)}(theta)/r + L_k^{(2)}(z)*L_k^{(0)}(z)*phi_i^{(2)}(r)*phi_i^{(0)}(r)*psi_j^{(2)}(theta)*psi_j^{(0)}(theta)/r)*C.b_theta + (-r*L_k^{(1)}(z)*L_k^{(0)}(z)*phi_i^{(1)}(r)*phi_i^{(0)}(r)*psi_j^{(1)}(theta)*psi_j^{(0)}(theta) + r*L_k^{(0)}(z)*L_k^{(1)}(z)*phi_i^{(0)}(r)*phi_i^{(1)}(r)*psi_j^{(0)}(theta)*psi_j^{(1)}(theta))*C.b_z

In [20]:
if C.dims == 3:
    c = Cross(v, C.b_r)
    display(c)
    display(c.doit())

Cross(v(x, y, z; V), C.b_r)

(L_k^{(2)}(z)*phi_i^{(2)}(r)*psi_j^{(2)}(theta)/r)*C.b_theta + (-r*L_k^{(1)}(z)*phi_i^{(1)}(r)*psi_j^{(1)}(theta))*C.b_z

In [21]:
if C.dims == 3:
    d = Curl(v)
    R = C.to_cartesian(d.doit())
    display(d)
    display(d.doit())
    display(R)

Curl(v(x, y, z; V))

((-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + L_k^{(2)}(z)*phi_i^{(2)}(r)*Derivative(psi_j^{(2)}(theta), theta))/r)*C.b_r + ((-L_k^{(2)}(z)*psi_j^{(2)}(theta)*Derivative(phi_i^{(2)}(r), r) + phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z))/r)*C.b_theta + ((r**2*L_k^{(1)}(z)*psi_j^{(1)}(theta)*Derivative(phi_i^{(1)}(r), r) + 2*r*L_k^{(1)}(z)*phi_i^{(1)}(r)*psi_j^{(1)}(theta) - L_k^{(0)}(z)*phi_i^{(0)}(r)*Derivative(psi_j^{(0)}(theta), theta))/r)*C.b_z

(-(-L_k^{(2)}(z)*psi_j^{(2)}(theta)*Derivative(phi_i^{(2)}(r), r) + phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z))*sin(theta) + (-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + L_k^{(2)}(z)*phi_i^{(2)}(r)*Derivative(psi_j^{(2)}(theta), theta))*cos(theta)/r)*R.i + ((-L_k^{(2)}(z)*psi_j^{(2)}(theta)*Derivative(phi_i^{(2)}(r), r) + phi_i^{(0)}(r)*psi_j^{(0)}(theta)*Derivative(L_k^{(0)}(z), z))*cos(theta) + (-r**2*phi_i^{(1)}(r)*psi_j^{(1)}(theta)*Derivative(L_k^{(1)}(z), z) + L_k^{(2)}(z)*phi_i^{(2)}(r)*Derivative(psi_j^{(2)}(theta), theta))*sin(theta)/r)*R.j + ((r**2*L_k^{(1)}(z)*psi_j^{(1)}(theta)*Derivative(phi_i^{(1)}(r), r) + 2*r*L_k^{(1)}(z)*phi_i^{(1)}(r)*psi_j^{(1)}(theta) - L_k^{(0)}(z)*phi_i^{(0)}(r)*Derivative(psi_j^{(0)}(theta), theta))/r)*R.k

In [22]:
from jaxfun.galerkin.arguments import VectorFunction

h = VectorFunction("h", C)
h

h(x, y, z)

In [23]:
H = h.doit()
H

(H_r(r, theta, z))*C.b_r + (H_\theta(r, theta, z))*C.b_theta + (H_z(r, theta, z))*C.b_z

In [24]:
u = TrialFunction(V, name="u")
uv = Dot(u, v)
srepr(uv)

"Dot(TrialFunction(x, y, z, Symbol('V')), TestFunction(x, y, z, Symbol('V')))"

In [25]:
UV = uv.doit()
UV

r**2*L_n^{(1)}(z)*L_k^{(1)}(z)*phi_l^{(1)}(r)*phi_i^{(1)}(r)*psi_m^{(1)}(theta)*psi_j^{(1)}(theta) + L_n^{(0)}(z)*L_k^{(0)}(z)*phi_l^{(0)}(r)*phi_i^{(0)}(r)*psi_m^{(0)}(theta)*psi_j^{(0)}(theta) + L_n^{(2)}(z)*L_k^{(2)}(z)*phi_l^{(2)}(r)*phi_i^{(2)}(r)*psi_m^{(2)}(theta)*psi_j^{(2)}(theta)

In [26]:
UV.args

(L_n^{(0)}(z)*L_k^{(0)}(z)*phi_l^{(0)}(r)*phi_i^{(0)}(r)*psi_m^{(0)}(theta)*psi_j^{(0)}(theta),
 L_n^{(2)}(z)*L_k^{(2)}(z)*phi_l^{(2)}(r)*phi_i^{(2)}(r)*psi_m^{(2)}(theta)*psi_j^{(2)}(theta),
 r**2*L_n^{(1)}(z)*L_k^{(1)}(z)*phi_l^{(1)}(r)*phi_i^{(1)}(r)*psi_m^{(1)}(theta)*psi_j^{(1)}(theta))

In [27]:
a = sp.separatevars(UV.args[1], dict=True, symbols=C._base_scalars)

In [28]:
a

{r: phi_l^{(2)}(r)*phi_i^{(2)}(r),
 theta: psi_m^{(2)}(theta)*psi_j^{(2)}(theta),
 z: L_n^{(2)}(z)*L_k^{(2)}(z),
 'coeff': 1}

In [29]:
import contextlib

for r in C._base_scalars:
    for j in a[r].args:
        with contextlib.suppress(AttributeError):
            print(j.local_index)

0
0
1
1
2
2


In [30]:
from flax import nnx

from jaxfun.operators import Cross, Curl, Div, Dot, Grad, Identity, Outer
from jaxfun.pinns import FlaxFunction, MLPSpace

V = MLPSpace([8], dims=C.dims, rank=1, system=C, name="V")  # Vector space for velocity
Q = MLPSpace([8], dims=C.dims, rank=0, system=C, name="Q")  # Scalar space for pressure

u = FlaxFunction(V, "u", rngs=nnx.Rngs(2002))
p = FlaxFunction(Q, "p", rngs=nnx.Rngs(1001))

In [31]:
C.simplify(Div(u).doit())

Derivative(u_r(r, theta, z), r) + Derivative(u_theta(r, theta, z), theta) + Derivative(u_z(r, theta, z), z) + u_r(r, theta, z)/r

In [32]:
R1 = Dot(Grad(u), u) - Div(Grad(u)) + Grad(p)
R1

-Div(Grad(u(x, y, z; V))) + Dot(Grad(u(x, y, z; V)), u(x, y, z; V)) + Grad(p(x, y, z; Q))

In [33]:
I = Identity(C)
R2 = Div(Outer(u, u)) - Div(
    (Grad(u) + Grad(u).T - sp.Rational(2, 3) * Div(u) * I) + p * I
)
R2.doit()

(r*(p(r, theta, z) - 2*Derivative(u_theta(r, theta, z), theta)/3 - 2*Derivative(u_z(r, theta, z), z)/3 - 2*(r*Derivative(u_r(r, theta, z), r) + u_r(r, theta, z))/(3*r) + 2*(Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z)/r)/r**2) - r*u_theta(r, theta, z)**2 + 2*u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_r(r, theta, z)*Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z)*Derivative(u_z(r, theta, z), z) + u_theta(r, theta, z)*Derivative(u_r(r, theta, z), theta) + u_z(r, theta, z)*Derivative(u_r(r, theta, z), z) - Derivative(p(r, theta, z), r) - 2*Derivative(u_r(r, theta, z), (r, 2)) - 2*Derivative(u_r(r, theta, z), (z, 2)) + 2*Derivative(u_theta(r, theta, z), r, theta)/3 + 2*Derivative(u_z(r, theta, z), r, z)/3 + 2*(r*Derivative(u_r(r, theta, z), (r, 2)) + 2*Derivative(u_r(r, theta, z), r))/(3*r) - (p(r, theta, z) + 2*Derivative(u_r(r, theta, z), r) - 2*Derivative(u_theta(r, theta, z), theta)/3 - 2*Derivative(u_z(r, theta, z), z)/3 - 2*(r*Derivative(u_r(r, thet

In [34]:
if C.dims == 3:
    R3 = Cross(Curl(u), u) + sp.S.Half * Grad(Dot(u, u)) - Div(Grad(u)) + Grad(p)
    display(R3)
    display(R3.doit())

Cross(Curl(u(x, y, z; V)), u(x, y, z; V)) - Div(Grad(u(x, y, z; V))) + Grad(p(x, y, z; Q)) + Grad(Dot(u(x, y, z; V), u(x, y, z; V)))/2

(r**2*u_theta(r, theta, z)*Derivative(u_theta(r, theta, z), r) + r*u_theta(r, theta, z)**2 + (Derivative(u_r(r, theta, z), z) - Derivative(u_z(r, theta, z), r))*u_z(r, theta, z) - (r**2*Derivative(u_theta(r, theta, z), r) + 2*r*u_theta(r, theta, z) - Derivative(u_r(r, theta, z), theta))*u_theta(r, theta, z) + u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_z(r, theta, z)*Derivative(u_z(r, theta, z), r) + Derivative(p(r, theta, z), r) - Derivative(u_r(r, theta, z), (r, 2)) - Derivative(u_r(r, theta, z), (z, 2)) + (Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z)/r)/r - Derivative(u_r(r, theta, z), r)/r - (-r*Derivative(u_theta(r, theta, z), theta) + Derivative(u_r(r, theta, z), (theta, 2)))/r**2)*C.b_r + (-Derivative(u_theta(r, theta, z), (r, 2)) - Derivative(u_theta(r, theta, z), (z, 2)) - 2*(Derivative(u_theta(r, theta, z), r) + u_theta(r, theta, z)/r)/r - Derivative(u_theta(r, theta, z), r)/r - (-r**2*Derivative(u_theta(r, theta, z), z) + Derivative(u_z(r, theta, z), t

In [35]:
ut = Dot(u, C.get_contravariant_basis_vector(1)).doit()
sp.srepr(ut)

"Function('u_theta')(r, theta, z)"

In [36]:
uu = Outer(u, u).doit()

In [37]:
uu

(u_r(r, theta, z)**2)*(C.b_r⊗C.b_r) + (u_r(r, theta, z)*u_theta(r, theta, z))*(C.b_r⊗C.b_theta) + (u_r(r, theta, z)*u_z(r, theta, z))*(C.b_r⊗C.b_z) + (u_r(r, theta, z)*u_theta(r, theta, z))*(C.b_theta⊗C.b_r) + (u_theta(r, theta, z)**2)*(C.b_theta⊗C.b_theta) + (u_theta(r, theta, z)*u_z(r, theta, z))*(C.b_theta⊗C.b_z) + (u_r(r, theta, z)*u_z(r, theta, z))*(C.b_z⊗C.b_r) + (u_theta(r, theta, z)*u_z(r, theta, z))*(C.b_z⊗C.b_theta) + (u_z(r, theta, z)**2)*(C.b_z⊗C.b_z)

In [38]:
C.simplify(Dot(R1, C.get_contravariant_basis_vector(0)).doit())

(r**2*(-(r*u_theta(r, theta, z) - Derivative(u_r(r, theta, z), theta))*u_theta(r, theta, z) + u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_z(r, theta, z)*Derivative(u_r(r, theta, z), z) + Derivative(p(r, theta, z), r) - Derivative(u_r(r, theta, z), (r, 2)) - Derivative(u_r(r, theta, z), (z, 2))) - r*Derivative(u_r(r, theta, z), r) + 2*r*Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z) - Derivative(u_r(r, theta, z), (theta, 2)))/r**2

In [39]:
C.simplify(R1.doit())

((r**2*(-(r*u_theta(r, theta, z) - Derivative(u_r(r, theta, z), theta))*u_theta(r, theta, z) + u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_z(r, theta, z)*Derivative(u_r(r, theta, z), z) + Derivative(p(r, theta, z), r) - Derivative(u_r(r, theta, z), (r, 2)) - Derivative(u_r(r, theta, z), (z, 2))) - r*Derivative(u_r(r, theta, z), r) + 2*r*Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z) - Derivative(u_r(r, theta, z), (theta, 2)))/r**2)*C.b_r + (u_r(r, theta, z)*Derivative(u_theta(r, theta, z), r) + u_theta(r, theta, z)*Derivative(u_theta(r, theta, z), theta) + u_z(r, theta, z)*Derivative(u_theta(r, theta, z), z) - Derivative(u_theta(r, theta, z), (r, 2)) - Derivative(u_theta(r, theta, z), (z, 2)) + 2*u_r(r, theta, z)*u_theta(r, theta, z)/r - 3*Derivative(u_theta(r, theta, z), r)/r + Derivative(p(r, theta, z), theta)/r**2 - Derivative(u_theta(r, theta, z), (theta, 2))/r**2 - 2*Derivative(u_r(r, theta, z), theta)/r**3)*C.b_theta + (u_r(r, theta, z)*Derivative(u_z(r, thet

In [40]:
r1 = Dot(R1, C.get_contravariant_basis_vector(0)).doit()
r1

(-r*u_theta(r, theta, z) + Derivative(u_r(r, theta, z), theta))*u_theta(r, theta, z) + u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_z(r, theta, z)*Derivative(u_r(r, theta, z), z) + Derivative(p(r, theta, z), r) - Derivative(u_r(r, theta, z), (r, 2)) - Derivative(u_r(r, theta, z), (z, 2)) + (Derivative(u_theta(r, theta, z), theta) + u_r(r, theta, z)/r)/r - Derivative(u_r(r, theta, z), r)/r - (-r*Derivative(u_theta(r, theta, z), theta) + Derivative(u_r(r, theta, z), (theta, 2)))/r**2

In [41]:
r1.expand()

-r*u_theta(r, theta, z)**2 + u_r(r, theta, z)*Derivative(u_r(r, theta, z), r) + u_theta(r, theta, z)*Derivative(u_r(r, theta, z), theta) + u_z(r, theta, z)*Derivative(u_r(r, theta, z), z) + Derivative(p(r, theta, z), r) - Derivative(u_r(r, theta, z), (r, 2)) - Derivative(u_r(r, theta, z), (z, 2)) - Derivative(u_r(r, theta, z), r)/r + 2*Derivative(u_theta(r, theta, z), theta)/r + u_r(r, theta, z)/r**2 - Derivative(u_r(r, theta, z), (theta, 2))/r**2